In [1]:
### This script is used to test local planner module.

In [ ]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from simworld.communicator.unrealcv import UnrealCV
from simworld.communicator.communicator import Communicator
from simworld.llm.a2a_llm import A2ALLM
from simworld.agent.humanoid import Humanoid
from simworld.utils.vector import Vector
from simworld.map.map import Map
from simworld.config.config_loader import Config
from simworld.local_planner.local_planner import LocalPlanner

In [2]:
os.environ['OPENAI_API_KEY'] = '<your_api_key>'

In [3]:
config = Config()

In [ ]:
# Initialize the LLM model
llm = A2ALLM(model_name='gpt-4o-mini', provider='openai')

In [ ]:
# Initialize the map
map = Map(config)
map.initialize_map_from_file()  # use default map
# Initialize the communicator
communicator = Communicator(UnrealCV())

In [7]:
# Initialize the humanoid agent
humanoid = Humanoid(position=Vector(0, 0), direction=Vector(1, 0), map=map, communicator=communicator, config=config)

In [8]:
# Initialize the local planner
local_planner = LocalPlanner(agent=humanoid, model=llm, rule_based=False)

In [ ]:
### Test parser
plan = 'Go to (1700, -1700) and pick up GEN_BP_Bottle_1_C.'
action_space = local_planner.parse(plan)

In [10]:
communicator.spawn_agent(humanoid, config['user.model_path'])

In [11]:
communicator.spawn_ue_manager(config['simworld.ue_manager_path'])

In [12]:
# Update humanoid position and direction in a separate thread
import threading
import time

def update(exit_event):
    while not exit_event.is_set():
        humanoid_ids = [humanoid.id]
        result = communicator.get_position_and_direction(humanoid_ids=humanoid_ids)
        for idx in humanoid_ids:
            pos, dir = result[('humanoid', idx)]
            humanoid.position = pos
            humanoid.direction = dir
        time.sleep(0.1)

exit_event = threading.Event()
t = threading.Thread(target=update, args=(exit_event,))
t.start()

In [ ]:
### Test executor
local_planner.execute(action_space)

In [14]:
exit_event.set()
t.join()